In [1]:
import numpy as np
import pandas as pd 
from importlib import reload
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx 
import json
import random 
import time

from src.Machine import Machine
from src.Job import Job
from src.Operation import Operation
from src.Agent import Agent
from src.Production_line import Production_line

def dsp(data, rows=10, columns=None):
    
    pd.options.display.max_rows = rows
    pd.options.display.max_columns = columns

    display(data)

    pd.options.display.max_rows = 15
    pd.options.display.max_columns = 20

In [2]:
prod_line = Production_line()
print(len(prod_line.create_action_space(14)))
agent = Agent(prod_line)
job1 = Job("TEST1", 1,20000)
job2 = Job("TEST2", 1,20000)
job3 = Job("TEST3", 1,20000)
job4 = Job("TEST4", 1,20000)
job5 = Job("TEST5", 1,20000)
job6 = Job("TEST6", 1,20000)
job7 = Job("TEST7", 1,20000)
job8 = Job("TEST8", 1,20000)
job9 = Job("TEST9", 1,20000)
job10 = Job("TEST10", 1,20000)
job11 = Job("TEST11", 1,20000)
job12 = Job("TEST12", 1,20000)
job13 = Job("TEST13", 1,20000)
job14 = Job("TEST14", 1,20000)

prod_line.add_job(job1)
prod_line.add_job(job2)
prod_line.add_job(job3)
prod_line.add_job(job4)
prod_line.add_job(job5)
prod_line.add_job(job6)
prod_line.add_job(job7)
prod_line.add_job(job8)
prod_line.add_job(job9)
prod_line.add_job(job10)
prod_line.add_job(job11)
prod_line.add_job(job12)
prod_line.add_job(job13)
prod_line.add_job(job14)


238


False

**to measure the execution time**

In [3]:
nbr = 1000

start = time.time()
for _ in range(nbr):
    agent.get_available_operation()
    
print('It took', (time.time()-start)/nbr, 'seconds.')


start = time.time()
for _ in range(nbr):
    agent.get_available_actions()
    
print('It took', (time.time()-start)/nbr, 'seconds.')



It took 4.388093948364258e-05 seconds.
It took 7.287192344665527e-05 seconds.


In [ ]:
start = time.time()
nbr = 1000
for i in range(nbr):
    agent.take_decision()
print('It took', (time.time()-start)/nbr, 'seconds.')

start = time.time()
agent.write_db('test.csv')
print('It took', (time.time()-start)/nbr, 'seconds.')

In [ ]:
planning = prod_line.get_gant_formated()

plan_df = pd.DataFrame()
for plan in planning:
    print(plan,"\n")
    df = pd.DataFrame(plan, columns =['Job','Machine', 'Operation', 'Start','Duration','Finish'])
    plan_df = pd.concat([plan_df, df], axis=0)

plan_df.reset_index(drop = True,inplace=True)

plan_df['Machine']= plan_df['Machine'].astype(str)
plan_df['Duration']= plan_df['Duration'].astype(int)




    
print(plan_df.dtypes)
print(plan_df)



In [ ]:
def visualize(results):
    
    operation_machine = {(1,1) : "broyage_poly_1",
                     (1,2) : "broyage_poly_2",
                     (2,3) : "tamisage_poly_3",
                     (3,4) : "mélange_4",
                     (3,5) : "mélange_5",
                     (4,6) : "extrusion_6",
                     (5,1) : "broyage_ug_1",
                     (5,2) : "broyage_ug_2",
                     (6,3) : "tamisage_ug_3",
                     (7,8) : "tween_8",
                     (8,7) : "combinaison_7",
                     (9,9) : "perry_9",
                     (10,10) : "sortie_lyo_10",
                     (11,11) : "capsulage_11",
                     (12,12) : "IV_12",
                     (13,13) : "envoi IRR_13",
                     (14,14) : "retour IRR_14"}
    
    schedule = results.copy()
    
    op_machine = []
    for row in schedule.iterrows():
        
        op = row[1]["Operation"]
        machine = row[1]["Machine"]
        op_machine.append(operation_machine[int(op),int(machine)])
    
    schedule["op_machine"] = op_machine
    
    JOBS = sorted(list(schedule['Job'].unique()))
    operation_machine_sorted = [value for key,value in operation_machine.items()][::-1]
    MACHINES = operation_machine_sorted
    makespan = schedule['Finish'].max()
    
    bar_style = {'alpha':1.0, 'lw':25, 'solid_capstyle':'butt'}
    text_style = {'color':'white', 'weight':'bold', 'ha':'center', 'va':'center'}
    colors = mpl.cm.Dark2.colors

    schedule.sort_values(by=['Job', 'Start'])
    schedule.set_index(['Job', 'op_machine'], inplace=True,append = True)
    
    fig, ax = plt.subplots(2,1, figsize=(12, 5+(len(JOBS)+len(MACHINES))/4))

    for jdx, j in enumerate(JOBS, 1):
        for mdx, m in enumerate(MACHINES, 1):
            for index,_,_ in schedule.index:
                if (index,j,m) in schedule.index:
                    

                    xs = schedule.loc[(index,j,m), 'Start']
                    xf = schedule.loc[(index,j,m), 'Finish']
                    ax[0].plot([xs, xf], [jdx]*2, c=colors[mdx%7], **bar_style)
                    ax[0].text((xs + xf)/2, jdx, m, **text_style)
                    ax[1].plot([xs, xf], [mdx]*2, c=colors[jdx%7], **bar_style)
                    ax[1].text((xs + xf)/2, mdx, j, **text_style)
                
    ax[0].set_title('Job Schedule')
    ax[0].set_ylabel('Job')
    ax[1].set_title('Machine Schedule')
    ax[1].set_ylabel('Machine')
    
    for idx, s in enumerate([JOBS, MACHINES]):
        ax[idx].set_ylim(0.5, len(s) + 0.5)
        ax[idx].set_yticks(range(1, 1 + len(s)))
        ax[idx].set_yticklabels(s)
        ax[idx].text(makespan, ax[idx].get_ylim()[0]-0.2, "{0:0.1f}".format(makespan), ha='center', va='top')
        ax[idx].plot([makespan]*2, ax[idx].get_ylim(), 'r--')
        ax[idx].set_xlabel('Time')
        ax[idx].grid(True)
        
    fig.tight_layout()
    
visualize(plan_df)